In [ ]:
import sys, os
cwd=os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, cwd)
import rasterio
import folium
import branca
from folium.plugins import HeatMap
import re
import branca.colormap as cmp
import pandas as pd
import geopandas as gpd
import shapefile as shp
from shapely.geometry import Point
from shapely.geometry.polygon import Point, Polygon
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr
import numpy as np
from scipy.spatial.distance import cdist
import utils.processing as pr
import utils.s3_utils as s3
import utils.plot as pl
from sklearn.preprocessing import PolynomialFeatures
from folium import plugins
import config.paths as path
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler, RobustScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score
import random
import utils.modeling as ml
from pathlib import Path

pd.set_option('display.max_columns', None)

region_code = 2
modelname = path.model_path

trainingdata_path = f'../../data/{modelname}/training/final/clustered/region_{region_code}/'
save_path=f'../../model/tree-models/final-tree-model-3/region-{region_code}/feature-selection/'

drop_features = 'y'
test_train = 'n' #Only set to 'y' after obtaining best training parameters using gridsearch.
test_split = .2
eval_method = 'precision'

# Import data
x = pd.read_csv(trainingdata_path + 'training_x/x.csv')
x = x.drop(['Unnamed: 0'],axis=1)
y = pd.read_csv(trainingdata_path + 'training_y/y.csv')
y = y.drop(['Unnamed: 0'],axis=1)
dummies_train = pd.read_csv(trainingdata_path + 'training_x/dummies.csv')
dummies_train = dummies_train.drop(['Unnamed: 0'],axis=1)

x.shape

#Remove pearson correlated features, while keeping the highest coeff feature in orig. model
if drop_features == 'y':
    drop_cols = path.xgboost_col_drop_dict[region_code]
    x = x.drop(drop_cols, axis=1)
    save_path=save_path
    Path(f"{save_path}").mkdir(parents=True, exist_ok=True)
else: 
    save_path=save_path
    Path(f"{save_path}").mkdir(parents=True, exist_ok=True)

In [ ]:
#initialize variables
nuts3_list = list(y.nuts_id.unique()) #will drop by 20% each run
nuts3_total_districts = len(nuts3_list)
random.seed(42)
drop_descriptor_cols=['nuts_id','nuts_name','longitude','latitude']
y_pred_list = []
if test_train == 'y':
    ### This section only runs if there is a saved clf model with best_params configured (produced in gridsearch section below)
    ### This will error out if you do not have saved clf model params. Please set test_train config to 'y' once params are set. 
        
    for i in range(0,int(1/test_split)):
        #randomly select test_split % of all nuts_ids, drop them from nuts 3 set space for next run
        #if-then clause makes sure we get the extra remaining nuts3 district from split
        if i == max(range(0,int(1/test_split))):
            test_nuts3_districts = nuts3_list
        else:
            test_nuts3_districts = random.sample(nuts3_list, int(nuts3_total_districts * test_split))
        nuts3_list = [x for x in nuts3_list if x not in list(test_nuts3_districts)]
        print(f'***NUTS3 IDs sampled for Run {i}: {test_nuts3_districts}***')
        print(nuts3_list)

        y_train = y[~y['nuts_id'].isin(test_nuts3_districts)]
        x_train = x[~x['nuts_id'].isin(test_nuts3_districts)]

        y_test = y[y['nuts_id'].isin(test_nuts3_districts)]
        x_test = x[x['nuts_id'].isin(test_nuts3_districts)]

        y_train = y_train.drop(drop_descriptor_cols, axis=1)
        x_train = x_train.drop(drop_descriptor_cols, axis=1)

        y_test = y_test.drop(['nuts_id','nuts_name'], axis=1)
        x_test = x_test.drop(drop_descriptor_cols, axis=1)

        final_param_dict = clf.best_params_    #for grid/randomsearchcv results
        regxGB, importances = ml.xgboost_model(x_train,y_train, params=final_param_dict)


        ### get roc-auc score by predicted using fitted model
        y_test['pred'] = regxGB.predict_proba(x_test)[:, 1]
        auroc_score = roc_auc_score(y_test['presence'], y_test['pred'])
        y_pred_list.append(y_test)
        
        print(f'AUROC for Test Run {i}:{auroc_score}')
        print(f'*** END OF RUN {i} ***')
        print('')

    print('***Cross-Validation Complete***\n')
    y_pred = pd.concat(y_pred_list)
    y_pred = pr.remove_or_combine_duplicates(y_pred, strategy='aggregate', aggfunc='mean')

    auroc_score = roc_auc_score(y_pred['presence'], y_pred['pred'])
    print(f'***AUROC for Cross-Validation:{auroc_score}***')
    ### output results

    param_text = f'''
    *** Region {region_code} Logistic Regression Results Summary
    ROC-AUC score: {auroc_score}
    features dropped: {drop_features}
    dropped cols: {drop_cols}
                        '''
    param_text2 = str(regxGB.get_params())

    with open(save_path+'AUROC-cv-summary.txt', "w") as text_file:
        text_file.write(param_text)
        text_file.write(param_text2)

    print('***Dropping descriptor columns from x and y dfs***')
    x = x.drop(drop_descriptor_cols, axis=1)
    y = y.drop(drop_descriptor_cols, axis=1)

if test_train == 'n':
    x = x.drop(drop_descriptor_cols, axis=1)
    y = y.drop(drop_descriptor_cols, axis=1)

### Model Validation / Tuning

### randomized search


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

In [ ]:
eval_method = 'precision'

# Initialize an XGBClassifier with a specific random seed for reproducibility
xgbr = xgb.XGBClassifier(seed=30)

# Define a dictionary of hyperparameters to tune
# - max_depth: Maximum tree depth for base learners
# - learning_rate: Boosting learning rate (xgb’s “eta”)
# - n_estimators: Number of trees to fit
# - colsample_bytree: Subsample ratio of columns when constructing each tree
# - min_split_loss: Minimum loss reduction required to make a further partition on a leaf node of the tree (gamma)
# - min_child_weight: Minimum sum of instance weight (hessian) needed in a child
# - subsample: Subsample ratio of the training instances
# - reg_lambda: L2 regularization term on weights (lambda)
# - reg_alpha: L1 regularization term on weights (alpha)
# - max_leaves: Maximum number of terminal nodes or leaves in a tree
# - eval_metric: Evaluation metrics for validation data, a default metric will be assigned according to objective (roc_auc for binary classification)
params = {'max_depth':list(np.arange(2, 8, step=1)) + [None],
          'n_estimators':np.arange(200, 800, step=100),
          'learning_rate': np.arange(.01, .02, step=.005),
          'colsample_bytree': np.arange(0.3, .8, step=.1),
          'min_child_weight': randint(0,1),
          'min_split_loss': [0],
          'subsample': np.arange(.4,1, step=.1),
          'reg_lambda': np.arange(0, 1.6, step=.2),
          'reg_alpha':  np.arange(0, 1.4, step=.2)
}

# Initialize GridSearchCV to find the best hyperparameters using cross-validation
# - estimator: The model to be tuned
# - param_grid: Dictionary with parameters names (str) as keys and lists of parameter settings to try as values
# - verbose: Controls the verbosity: the higher, the more messages
# - n_jobs: Number of jobs to run in parallel (-1 means using all processors)
# - scoring: Strategy to evaluate the performance of the cross-validated model on the test set
clf = RandomizedSearchCV(estimator=xgbr,
                   param_distributions=params,
                   n_iter=1500,
                   verbose=2, 
                   n_jobs=-1, 
                   scoring=eval_method,
                   cv=5)

# Fit the GridSearchCV model
clf.fit(x, y)

# Print the best hyperparameters found by GridSearchCV
print("Best parameters:", clf.best_params_)

# Printing best AUC score
print(f'precision is: {clf.best_score_}')


In [ ]:
# Print the best hyperparameters found by GridSearchCV
print(clf.best_params_)

# Printing best AUC score
print(f'precision is: {clf.best_score_}')

### Gridsearch


In [ ]:

eval_method = 'precision'

# Initialize an XGBClassifier with a specific random seed for reproducibility
xgbr = xgb.XGBClassifier(seed=30)

# Define a dictionary of hyperparameters to tune
# - max_depth: Maximum tree depth for base learners
# - learning_rate: Boosting learning rate (xgb’s “eta”)
# - n_estimators: Number of trees to fit
# - colsample_bytree: Subsample ratio of columns when constructing each tree
# - min_split_loss: Minimum loss reduction required to make a further partition on a leaf node of the tree (gamma)
# - min_child_weight: Minimum sum of instance weight (hessian) needed in a child
# - subsample: Subsample ratio of the training instances
# - reg_lambda: L2 regularization term on weights (lambda)
# - reg_alpha: L1 regularization term on weights (alpha)
# - max_leaves: Maximum number of terminal nodes or leaves in a tree
# - eval_metric: Evaluation metrics for validation data, a default metric will be assigned according to objective (roc_auc for binary classification)
params = {
    'colsample_bytree': [.5,.6],
    'learning_rate': [.012,.015],
    'max_depth': [2,3,4],
    'min_child_weight': [0],
    'min_split_loss': [0],
    'n_estimators': [600,700,800],
    'reg_alpha':  [0,.1,.2],
    'reg_lambda': [1,1.2],
    'subsample': [.4,.5]
}


# Initialize GridSearchCV to find the best hyperparameters using cross-validation
# - estimator: The model to be tuned
# - param_grid: Dictionary with parameters names (str) as keys and lists of parameter settings to try as values
# - verbose: Controls the verbosity: the higher, the more messages
# - n_jobs: Number of jobs to run in parallel (-1 means using all processors)
# - scoring: Strategy to evaluate the performance of the cross-validated model on the test set
clf = GridSearchCV(estimator=xgbr,
                   param_grid=params,
                   verbose=2, n_jobs=-1, scoring=eval_method,cv=5)

# Fit the GridSearchCV model
clf.fit(x, y)

# Print the best hyperparameters found by GridSearchCV
print("Best parameters:", clf.best_params_)

# Printing best AUC score
print(f'precision is: {clf.best_score_}')


### Check results

In [ ]:
print(f'precision is: {clf.best_score_}')

In [ ]:
clf.best_params_

In [ ]:
#save params
param_text = str(clf.best_params_)
with open(save_path+'model_params_gridsearch.txt', "w") as text_file:
   text_file.write(str(clf.best_score_))
   text_file.write(param_text)

## final training model

In [ ]:
final_param_dict = clf.best_params_    #for grid/randomsearchcv results
param_text = str(clf.best_params_)

In [ ]:
# Run full model using all data points for training
regxGB, importances= ml.xgboost_model(x,y, params=final_param_dict)

# view and save feature importance results
display(importances.head(15))
save_name=f"feature_importances.csv"
importances.to_csv(save_path+save_name)

In [ ]:
### in-sample testing
regxGB, importances= ml.xgboost_model(x,y, params=final_param_dict)
y_pred = (regxGB.predict_proba(x)[:,1]).round(3)

auroc_score = roc_auc_score(y['presence'], y_pred)
print(f'***AUROC for Cross-Validation:{auroc_score}***')

param_text = f'''
*** Region {region_code} Logistic Regression Results Summary
ROC-AUC score: {auroc_score}
features dropped: {drop_features}
dropped cols: {drop_cols}
                    '''
param_text2 = str(regxGB.get_params())
with open(save_path+'AUROC-insample-summary.txt', "w") as text_file:
    text_file.write(param_text)
    text_file.write(param_text2)

## Test Predictions on Cov Data

### Test prediction on Covariates only

In [ ]:
fn = f'../../data/{modelname}/processed-predictor-parquets/clustered/{region_code}-predictors.parquet'
df = pd.read_parquet(fn, engine='pyarrow')

In [ ]:
# Test Predictions
#get predictors for all countries
test_list = []
sample_sizer = 5

fn = f'../../data/{modelname}/processed-predictor-parquets/clustered/{region_code}-predictors.parquet'
df = pd.read_parquet(fn, engine='pyarrow')
samp_size = df.shape[0]/sample_sizer
print(f'sample_size of {region_code} : {samp_size}')

df = df.sample(int(np.round(samp_size)), random_state=42)
test_list.append(df)

test = pd.concat(test_list)

del df, test_list


test_df = test.drop(['geometry','landcover'],axis=1)
test_df.columns = map(str.lower, test_df.columns)

#get dummies for landcover 
dummies_test = pd.get_dummies(test_df['cat'])
#add dummy categoricals to match dummies_train used in training
for missing_env in set(dummies_train.columns).difference(set(dummies_test.columns)):
    dummies_test[missing_env] = 0


test_df2 = test_df.drop('cat',axis=1)
test_df2 = pd.concat([test_df2, dummies_test], axis = 1).reset_index().drop('index',axis=1)

#Match and Re-order columns to the same as was used in training dataframe x
x_test = x.rename(columns = {'tg-grp-mean-days-above-5degC-monthly-ratio':'tg-grp-mean-days-above-5degc-monthly-ratio'})
nolatlon = test_df2[x_test.columns]


x_test = nolatlon
x_test = x_test.rename(columns = {'tg-grp-mean-days-above-5degc-monthly-ratio':'tg-grp-mean-days-above-5degC-monthly-ratio'})

print('Running prediction')
test_pred = pd.DataFrame()
test_pred['pred'] = (regxGB.predict_proba(x_test)[:,1]).round(3)
test_pred['latitude'] = test_df2['lat_env']
test_pred['longitude'] = test_df2['lon_env']

print(test_pred.info())

print('Outputting prediction data')
print('removing duplicate columns')
test_pred = pr.remove_or_combine_duplicates(test_pred, strategy='aggregate', aggfunc='mean')

del  nolatlon, test_df2, test_df, dummies_test, x_test

In [ ]:
test_pred.to_csv(save_path+f'test-predictions.csv')

## Plotting Predictions

In [ ]:

hist = test_pred.pred.hist(bins=100)
fig = hist.figure
fig.savefig(save_path+'distribution_histogram.png')

### Dot Plot and Choropleth Map

In [ ]:
country_codes = ['DK','NO','SE','FI','AT','CH','CZ','DE','EE','FR', 'LT','LV','NL','PL','SK', 'IT','UK','HR','BE','SI','LU']

m, ma = pl.plot_xgboost_maps(test_pred, country_codes,region=region_code, dot_sample=50000)

In [ ]:
save_name=f"xgb-dotmap.html"
m.save(save_path+save_name)

save_name=f"xgb-choropleth.html"
ma.save(save_path+save_name)


## Permutation Importance 

In [ ]:
region_code = 0
modelname = path.model_path

trainingdata_path = f'../../data/{modelname}/training/final/clustered/region_{region_code}/'
save_path=f'../../model/tree-models/final-tree-model-2/region-{region_code}/feature-selection/permutation_test/'

drop_features = 'y'
test_split = .2

# Import data
x = pd.read_csv(trainingdata_path + 'training_x/x.csv')
x = x.drop(['Unnamed: 0'],axis=1)
y = pd.read_csv(trainingdata_path + 'training_y/y.csv')
y = y.drop(['Unnamed: 0'],axis=1)
dummies_train = pd.read_csv(trainingdata_path + 'training_x/dummies.csv')
dummies_train = dummies_train.drop(['Unnamed: 0'],axis=1)
drop_descriptor_cols=['nuts_id','nuts_name','longitude','latitude']

x.shape

#Remove pearson correlated features, while keeping the highest coeff feature in orig. model
if drop_features == 'y':
    drop_cols = path.xgboost_col_drop_dict[region_code]
    x = x.drop(drop_cols, axis=1)
    save_path=save_path
    Path(f"{save_path}").mkdir(parents=True, exist_ok=True)
else: 
    save_path=save_path
    Path(f"{save_path}").mkdir(parents=True, exist_ok=True)

In [ ]:
landcover_list = [col for col in x.columns if 'cat_' in col]

In [ ]:
AUROC_degradation = {}
base_AUROC = 0.728190967910594
final_param_dict = {'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.3, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.012, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 9, 'max_leaves': None, 'min_child_weight': 0, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 250, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': 0.8, 'reg_lambda': 1.5, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.5, 'tree_method': None, 'validate_parameters': None, 'verbosity': None, 'min_split_loss': 0}
n = 10

random.seed(42)

for cov in set(x.columns).difference(set(landcover_list)).difference(set(drop_descriptor_cols)):
    sum_auroc =0
    print(f'***Run {cov}***')
    for j in range(n):
        x_shuffled = x.copy()
        x_shuffled[cov] = np.random.permutation(x[cov].values)

        ### begin cv
        nuts3_list = list(y.nuts_id.unique()) #will drop by 20% each run
        nuts3_total_districts = len(nuts3_list)
        random.seed(42)
        drop_descriptor_cols=['nuts_id','nuts_name','longitude','latitude']
        y_pred_list = []
        for i in range(0,int(1/test_split)):
            #randomly select test_split % of all nuts_ids, drop them from nuts 3 set space for next run
            #if-then clause makes sure we get the extra remaining nuts3 district from split
            if i == max(range(0,int(1/test_split))):
                test_nuts3_districts = nuts3_list
            else:
                test_nuts3_districts = random.sample(nuts3_list, int(nuts3_total_districts * test_split))
            nuts3_list = [x for x in nuts3_list if x not in list(test_nuts3_districts)]

            y_train = y[~y['nuts_id'].isin(test_nuts3_districts)]
            x_train = x_shuffled[~x_shuffled['nuts_id'].isin(test_nuts3_districts)]

            y_test = y[y['nuts_id'].isin(test_nuts3_districts)]
            x_test = x[x_shuffled['nuts_id'].isin(test_nuts3_districts)]

            y_train = y_train.drop(drop_descriptor_cols, axis=1)
            x_train = x_train.drop(drop_descriptor_cols, axis=1)

            y_test = y_test.drop(['nuts_id','nuts_name'], axis=1)
            x_test = x_test.drop(drop_descriptor_cols, axis=1)

            regxGB, importances = ml.xgboost_model(x_train,y_train, params=final_param_dict)

            ### get roc-auc score by predicted using fitted model
            y_test['pred'] = regxGB.predict_proba(x_test)[:, 1]
            auroc_score = roc_auc_score(y_test['presence'], y_test['pred'])
            y_pred_list.append(y_test)

        y_pred = pd.concat(y_pred_list)
        y_pred = pr.remove_or_combine_duplicates(y_pred, strategy='aggregate', aggfunc='mean')

        auroc_score = roc_auc_score(y_pred['presence'], y_pred['pred'])
        sum_auroc += auroc_score
        
    avg_auroc = sum_auroc / n #average of all n runs
    print(f'***AUROC for n=10 {cov} CV:{avg_auroc}***')

    AUROC_degradation[cov] = base_AUROC - avg_auroc
    

In [ ]:
### Remove landcover variables
sum_auroc = 0
for i in range(n):
    x_shuffled = x.copy()
    for cov in landcover_list:
        x_shuffled[cov] = np.random.permutation(x[cov].values)
        print(f'Shuffled values in {cov}')

    ### begin cv
    nuts3_list = list(y.nuts_id.unique()) #will drop by 20% each run
    nuts3_total_districts = len(nuts3_list)
    drop_descriptor_cols=['nuts_id','nuts_name','longitude','latitude']
    y_pred_list = []
    for i in range(0,int(1/test_split)):
        #randomly select test_split % of all nuts_ids, drop them from nuts 3 set space for next run
        #if-then clause makes sure we get the extra remaining nuts3 district from split
        if i == max(range(0,int(1/test_split))):
            test_nuts3_districts = nuts3_list
        else:
            test_nuts3_districts = random.sample(nuts3_list, int(nuts3_total_districts * test_split))
        nuts3_list = [x for x in nuts3_list if x not in list(test_nuts3_districts)]

        y_train = y[~y['nuts_id'].isin(test_nuts3_districts)]
        x_train = x_shuffled[~x_shuffled['nuts_id'].isin(test_nuts3_districts)]

        y_test = y[y['nuts_id'].isin(test_nuts3_districts)]
        x_test = x[x_shuffled['nuts_id'].isin(test_nuts3_districts)]

        y_train = y_train.drop(drop_descriptor_cols, axis=1)
        x_train = x_train.drop(drop_descriptor_cols, axis=1)

        y_test = y_test.drop(['nuts_id','nuts_name'], axis=1)
        x_test = x_test.drop(drop_descriptor_cols, axis=1)

        regxGB, importances = ml.xgboost_model(x_train,y_train, params=final_param_dict)

        ### get roc-auc score by predicted using fitted model
        y_test['pred'] = regxGB.predict_proba(x_test)[:, 1]
        auroc_score = roc_auc_score(y_test['presence'], y_test['pred'])
        y_pred_list.append(y_test)
        

    y_pred = pd.concat(y_pred_list)
    y_pred = pr.remove_or_combine_duplicates(y_pred, strategy='aggregate', aggfunc='mean')

    auroc_score = roc_auc_score(y_pred['presence'], y_pred['pred'])
    print(f'***AUROC for Cross-Validation:{auroc_score}***')
    sum_auroc += auroc_score

avg_auroc = sum_auroc / n #average of all n runs
print(f'***AUROC for n=10 landcover CV:{avg_auroc}***')

AUROC_degradation['landcover'] = base_AUROC - avg_auroc


print(AUROC_degradation)

In [ ]:
auroc_degradation_df = pd.DataFrame(AUROC_degradation.items(), columns=['cov','loss']).sort_values('loss', ascending=False)
auroc_degradation_df

In [ ]:
auroc_degradation_df.to_csv(save_path+'permutation-importance.csv')